In [221]:
from backend.dsm2_reader import read_scenario_dir
import pandas as pd
import altair as alt
from data_config import gatef, elev_list, flow_list, stn_name
import numpy as np


In [80]:
stn_list = ['MID_GATEOP','GLC_GATEOP','OLD_GATEOP']

In [7]:
data = read_scenario_dir("C:/Users/Inigo/Projects/sdg-dashboard/data", v7_filter="_7")

mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_SDG_7.dss
mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_SDG_7.dss
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.


In [28]:
print(data.keys())

dict_keys(['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'])


In [245]:
def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

def filter_data(data, column, values, start_date=None, end_date=None):
    """Filter data based on a column and a list of values."""
    if start_date:
        data = data[data[column].isin(values)].dropna()
        data = data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]
    else:
        data = data[data[column].isin(values)].dropna()
    return data

def rename_gates(data, column, rename_map):
    """Rename values in a column in the DataFrame."""
    data[column] = data[column].replace(rename_map)
    return data

def set_datetime_index(data):
    data.set_index('datetime', inplace=True)
    data = data['value']
    return(data)


# def filter_date(data, start_date, end_date):
#     return data[(data['datetime'] >= start_date) & (data['datetime'] <= end_date)]

# def get_gate_up(data, column)

def prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef):
    """
    Prepare gate data dictionary from input datasets.
    
    Parameters:
    - sdg_flow: DataFrame with flow data.
    - sdg_stage: DataFrame with stage data.
    - gatop: DataFrame with gate operation data.
    - hydro_wl: Dataframe with water level data.
    - gatef: DataFrame containing gate configuration.

    Returns:
    - Dictionary containing gate data.
    """
    full_data = {}
    for i, gate_name in enumerate(gatef['name']):
        flow_data = filter_data(sdg_flow, 'gate_op', [gatef["flow_op"][i]])
        gate_data = filter_data(sdg_stage, 'gate_op', [gatef['gate_status'][i]])
        gate_data = gate_data[gate_data['unit'] == "FEET"]
        gateop_data = sdg_gateop[sdg_gateop['gate_op']==gatef['station'][i]]
        hydro_wl_data = hydro_wl[hydro_wl['gate']==gatef['station'][i]]
        full_data[gatef["ID"][i]] = {
            'name': gatef['name'][i],
            'bottom_elev': gatef['bottom_elev'][i],
            'width': gatef['width'][i],
            'flow_data': flow_data, #sc2_flow
            'gate_data': gate_data, #sc2_gate
            'gate_operation_data': gateop_data, #sc2_gateop
            "water_level_data": hydro_wl_data #sc2_wl
            
        }
    return full_data

def generate_full_model_data(data, path, gatef, elev_list, flow_list, stn_name, stn_list, start_date=None, end_date=None):
    sdg = data[path]['sdg']
    hydro = data[path]['hydro']

    sdg_stage = filter_data(sdg, 'gate_op', elev_list, start_date, end_date)
    sdg_flow = filter_data(sdg, 'gate_op', flow_list, start_date, end_date)
    sdg_gateop = filter_data(sdg, 'gate_op', stn_list, start_date, end_date)
    gate_names = {'MID_GATEOP': 'MHO', 'GLC_GATEOP': 'DGL', 'OLD_GATEOP': 'OLD'}
    sdg_gateop = rename_gates(sdg_gateop, "gate_op", gate_names)
    
    hydro_wl = hydro[hydro['parameter']=="STAGE"]
    hydro_wl = filter_data(hydro_wl, 'gate', stn_name, start_date, end_date)

    full_data = prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef)
    
    sdg_flow_GLC_FLOW_FISH = set_datetime_index(full_data['GLC']['flow_data'])
    sdg_flow_GLC_GATE_UP = set_datetime_index(full_data['GLC']['gate_data'])
    glc_bottom_elev = full_data['GLC']['bottom_elev']
    glc_width = full_data['GLC']['width']


    sdg_flow_MID_FLOW_FISH = set_datetime_index(full_data['MID']['flow_data'])
    sdg_flow_MID_GATE_UP = set_datetime_index(full_data['MID']['gate_data'])
    mid_bottom_elev = full_data['MID']['bottom_elev']
    mid_width = full_data['MID']['width']
    
    sdg_flow_OLD_FLOW_FISH = set_datetime_index(full_data['OLD']['flow_data'])
    sdg_flow_OLD_GATE_UP = set_datetime_index(full_data['OLD']['gate_data'])
    old_bottom_elev = full_data['OLD']['bottom_elev']
    old_width = full_data['OLD']['width']
    
    full_data["GLC"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width))
    full_data["MID"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width))
    full_data["OLD"]['vel'] = pd.DataFrame(calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width))

    for key in ["GLC", "MID", "OLD"]:
        full_data[key]['vel']['datetime'] = full_data[key]['vel'].index
        full_data[key]['vel'] = full_data[key]['vel'].reset_index(drop=True)
        full_data[key]['vel'] = full_data[key]['vel'][["datetime", "value"]]
    
    return full_data


In [246]:
start_zoom = '2016-05-01'
end_zoom = '2016-11-30'
fpv1ma_hydro_full_data = generate_full_model_data(data, 
                 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 
                 gatef,
                 elev_list,
                 flow_list,
                 stn_name,
                 stn_list,
                 start_zoom,
                 end_zoom
                 )

fpv2ma_hydro_full_data = generate_full_model_data(data,
                              'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss',
                              gatef,
                              elev_list,
                              flow_list,
                              stn_name,
                              stn_list,
                              start_zoom,
                            end_zoom)

In [247]:

fpv2ma_hydro_full_data["OLD"]["vel"]

,datetime,value
0,2016-05-01 00:00:00,-0.540797
1,2016-05-01 00:15:00,-0.582787
2,2016-05-01 00:30:00,-0.613909
3,2016-05-01 00:45:00,-0.598160
4,2016-05-01 01:00:00,-0.601773
...,...,...
20444,2016-11-29 23:00:00,9.362068
20445,2016-11-29 23:15:00,9.657503
20446,2016-11-29 23:30:00,9.941912
20447,2016-11-29 23:45:00,10.228580


In [255]:
def post_process_gateop(model_data, gate):
    gate_up_df = model_data[gate]['gate_operation_data'][["datetime", "value"]]
    gate_up_df["gate_status"] = gate_up_df['value']>=10
    gate_up_df['consecutive_groups'] = (gate_up_df['value'] != gate_up_df['value'].shift()).cumsum()
    gate_up_df['min_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('min')
    gate_up_df['max_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('max')
    consecutive_streaks = gate_up_df.groupby(['consecutive_groups', 'value', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
    consecutive_streaks['streak_duration'] = consecutive_streaks['count'] * 15 / 60
    consecutive_streaks_clean = consecutive_streaks.drop(['value', 'consecutive_groups', 'max_datetime'], axis = 1)
    # print(consecutive_streaks_clean.head())
    merged_gate_df = pd.merge(gate_up_df, consecutive_streaks_clean,left_on="min_datetime", right_on="min_datetime")
    merged_gate_df = merged_gate_df.drop(['consecutive_groups', 'value'], axis=1)
    merged_gate_df = merged_gate_df.rename(columns={"min_datetime": "gate_min_datetime", 
                                                "max_datetime": "gate_max_datetime",
                                                "count": "gate_count",
                                                "streak_duration": "gate_streak_duration"})
    return merged_gate_df

def post_process_velocity(model_data, gate):
    vel_zoom_df =model_data[gate]["vel"]
    vel_zoom_df['Velocity_Category'] = np.where(vel_zoom_df['value'] >= 8, "Over 8ft/s", "Under 8ft/s")
    #.shift shift value down and compare each value with the previous row; increase value when rows are different
    vel_zoom_df['consecutive_groups'] = (vel_zoom_df['Velocity_Category'] != vel_zoom_df['Velocity_Category'].shift()).cumsum()
    vel_zoom_df['min_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('min')
    vel_zoom_df['max_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('max')
    vel_zoom_df['date'] = vel_zoom_df['datetime'].dt.date.astype(str)
    consecutive_streaks_vel = vel_zoom_df.groupby(['consecutive_groups', 'Velocity_Category', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
    consecutive_streaks_vel['streak_duration'] = consecutive_streaks_vel['count'] * 15 / 60

    consecutive_streaks_vel_clean = consecutive_streaks_vel.drop(['consecutive_groups', 'Velocity_Category', 'max_datetime'], axis=1)
    merged_vel_df = pd.merge(vel_zoom_df, consecutive_streaks_vel_clean,left_on="min_datetime", right_on="min_datetime")

    return merged_vel_df

def post_process_full_data(model_data, gate):
    merged_gate_df = post_process_gateop(model_data, gate)
    merged_vel_df = post_process_velocity(model_data, gate)
    full_merged_df = pd.merge(merged_vel_df, merged_gate_df, left_on="datetime", right_on="datetime")
    full_merged_df['time_unit'] = 0.25
    
    return full_merged_df

In [256]:
full_data = post_process_full_data(fpv2ma_hydro_full_data, "GLC")
print(full_data.head())


             datetime     value Velocity_Category  consecutive_groups  \
0 2016-05-01 00:00:00 -0.394567       Under 8ft/s                   1   
1 2016-05-01 00:15:00 -0.444881       Under 8ft/s                   1   
2 2016-05-01 00:30:00 -0.452719       Under 8ft/s                   1   
3 2016-05-01 00:45:00 -0.467519       Under 8ft/s                   1   
4 2016-05-01 01:00:00 -0.465833       Under 8ft/s                   1   

  min_datetime        max_datetime        date  count  streak_duration  \
0   2016-05-01 2016-05-01 05:00:00  2016-05-01     21             5.25   
1   2016-05-01 2016-05-01 05:00:00  2016-05-01     21             5.25   
2   2016-05-01 2016-05-01 05:00:00  2016-05-01     21             5.25   
3   2016-05-01 2016-05-01 05:00:00  2016-05-01     21             5.25   
4   2016-05-01 2016-05-01 05:00:00  2016-05-01     21             5.25   

   gate_status gate_min_datetime   gate_max_datetime  gate_count  \
0        False        2016-05-01 2016-05-01 03:4

C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\2315908888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gate_up_df["gate_status"] = gate_up_df['value']>=10
C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\2315908888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gate_up_df['consecutive_groups'] = (gate_up_df['value'] != gate_up_df['value'].shift()).cumsum()


In [249]:
# gate_up = post_process_gateop(fpv2ma_hydro_full_data, "MID")
vel_zoom_df =fpv2ma_hydro_full_data["GLC"]["vel"]
# vel_zoom_df['datetime'] = vel_zoom_df.index
# vel_zoom_df = vel_zoom_df.reset_index(drop=True) 
# print(gate_up.head()) # fpv2ma_hydro_full_data['GLC']['gate_operation_data']
vel_zoom_df['Velocity_Category'] = np.where(vel_zoom_df['value'] >= 8, "Over 8ft/s", "Under 8ft/s")
#.shift shift value down and compare each value with the previous row; increase value when rows are different
vel_zoom_df['consecutive_groups'] = (vel_zoom_df['Velocity_Category'] != vel_zoom_df['Velocity_Category'].shift()).cumsum()
# print(vel_zoom_df)
vel_zoom_df['min_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('min')
vel_zoom_df['max_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('max')
vel_zoom_df['date'] = vel_zoom_df['datetime'].dt.date.astype(str)
consecutive_streaks_vel = vel_zoom_df.groupby(['consecutive_groups', 'Velocity_Category', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
consecutive_streaks_vel['streak_duration'] = consecutive_streaks_vel['count'] * 15 / 60

consecutive_streaks_vel_clean = consecutive_streaks_vel.drop(['consecutive_groups', 'Velocity_Category', 'max_datetime'], axis=1)
merged_df = pd.merge(vel_zoom_df, consecutive_streaks_vel_clean,left_on="min_datetime", right_on="min_datetime")
# print(merged_df.head(5))
# over_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Over 8ft/s']
# under_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Under 8ft/s']

# # Total duration for over and under 8ft/s
# total_over_duration = over_streaks['count'].sum()
# total_under_duration = under_streaks['count'].sum()

# over_streak_duration =  over_streaks['count'] * 15 / 60
# under_streak_duration = under_streaks['count'] * 15 / 60

# # Group by the Velocity Category and count the number of days in each category
# vel_count_df = vel_zoom_df.groupby('Velocity_Category').size().reset_index(name='count')

# total_time_df = pd.DataFrame({
#     'Velocity_Category': ['Over 8ft/s', 'Under 8ft/s'],
#     'Total Hours': [total_over_duration * 15 / 60, total_under_duration * 15 / 60]
# })

# consecutive_streaks_vel.rename(columns={
#     "Velocity_Category": "Status"},
#     inplace = True)

In [253]:

print(consecutive_streaks_vel_clean.head())

         min_datetime  count  streak_duration
0 2016-05-01 00:00:00     21             5.25
1 2016-05-01 05:15:00     19             4.75
2 2016-05-01 10:00:00     22             5.50
3 2016-05-01 15:30:00     31             7.75
4 2016-05-01 23:15:00     24             6.00


In [ ]:
sdg = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['sdg']
sdg_stage = sdg[sdg['gate_op'].isin(elev_list)]
sdg_flow = sdg[sdg['gate_op'].isin(flow_list)]
sdg_gateop = sdg[sdg['gate_op'].isin(stn_list)]
gate_names = {'MID_GATEOP':'MHO',
              'GLC_GATEOP':'DGL',
              'OLD_GATEOP':'OLD'}
sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)
print(sdg_gateop.head())
year = "2012"

# sdg_gateop =filter_date(sdg_gateop, start_zoom, end_zoom)
print(sdg_gateop.head())
hydro = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['hydro']
hydro_wl = hydro[hydro['parameter']=="STAGE"]
hydro_wl = hydro_wl[hydro_wl['gate'].isin(stn_name)]
# print(hydro_wl.head())

# print(wl.head())
# print(data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss'].keys())
# # hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.
# print(hydro_gateop['gate'].unique())
# print(hydro_gateop[hydro_gateop['gate']=="DGL"])

C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\599939783.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)


             datetime  value  unit gate_op scenario
0 2015-12-31 00:15:00    NaN  FEET     DGL   FPV1MA
1 2015-12-31 00:30:00    NaN  FEET     DGL   FPV1MA
2 2015-12-31 00:45:00    NaN  FEET     DGL   FPV1MA
3 2015-12-31 01:00:00    NaN  FEET     DGL   FPV1MA
4 2015-12-31 01:15:00    NaN  FEET     DGL   FPV1MA
                 datetime     value  unit        gate_op scenario
11711 2016-05-01 00:00:00  3.680600  FEET  GLC_GATE_DOWN   FPV1MA
11712 2016-05-01 00:15:00  3.818419  FEET  GLC_GATE_DOWN   FPV1MA
11713 2016-05-01 00:30:00  3.946771  FEET  GLC_GATE_DOWN   FPV1MA
11714 2016-05-01 00:45:00  4.091312  FEET  GLC_GATE_DOWN   FPV1MA
11715 2016-05-01 01:00:00  4.218915  FEET  GLC_GATE_DOWN   FPV1MA


In [208]:
full_data = {}

for i, gate_name in enumerate(gatef['name']):

    flow_data = sdg_flow[sdg_flow.gate_op == gatef["flow_op"][i]]
    gate_data = sdg_stage[sdg_stage.gate_op== gatef['gate_status'][i]]
     
    # Drop rows with NaN values from flow_data and gate_data
    flow_data = flow_data.dropna()
    gate_data = gate_data.dropna()
    
    full_data[gatef["ID"][i]] = {
        'name': gatef['name'][i],
        'bottom_elev': gatef['bottom_elev'][i],
        'width': gatef['width'][i],
        'flow_data': flow_data,
        'gate_data': gate_data
    }
sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']
sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']
glc_bottom_elev = full_data['GLC']['bottom_elev']
glc_width = full_data['GLC']['width']



In [196]:
sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
mid_bottom_elev = full_data['MID']['bottom_elev']
mid_width = full_data['MID']['width']

sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
old_bottom_elev = full_data['OLD']['bottom_elev']
old_width = full_data['OLD']['width']

In [209]:
thr_vel = 8
# mask = (sdg_flow.index.month>=5) & (sdg_flow.index.month<12) # time when gate are in operation
# vel=pd.DataFrame()
sdg_flow_GLC_FLOW_FISH.set_index('datetime', inplace=True)
sdg_flow_GLC_FLOW_FISH = sdg_flow_GLC_FLOW_FISH['value']
# print(sdg_flow_GLC_FLOW_FISH.head())
# sdg_flow_GLC_GATE_UP.set_index('datetime', inplace=True)
# sdg_flow_GLC_GATE_UP = sdg_flow_GLC_GATE_UP['value']
# sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
vel = pd.DataFrame()
full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
# full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
# full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

# vel['MID'] = calc_vel(sdg_flow.MID_FLOW_FISH,sdg_flow.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
# vel['OLD'] = calc_vel(sdg_flow.OLD_FLOW_FISH,sdg_flow.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )
# print(full_data["GLC"]['vel'].tail())
print(full_data.keys())

TypeError: Addition/subtraction of integers and integer-arrays with DatetimeArray is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`